In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from My_Loss import HardTripletLoss
from My_Loss import HardTripletLoss2
from My_Loss import HardTripletLoss_D
from tensorboardX import SummaryWriter

In [2]:
BATCH_SIZE = 64
EPISODE = 100000
TEST_EPISODE = 1000
LEARNING_RATE =2e-5
Weight_Deacy = 1e-6
GPU = 0
Margin = 3

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'CUB1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import MetricNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(312,1200,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1200,2048)
#triplet_network = TripletNetwork(attribute_network, metric_network)  #metric learning   
attribute_network.cuda(GPU) 
metric_network.cuda(GPU)
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
metric_network_optim = torch.optim.Adam(metric_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)

attribute_network_scheduler = StepLR(attribute_network_optim , step_size=10000 , gamma=1)
metric_network_scheduler = StepLR(metric_network_optim , step_size=10000 , gamma=1)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024

        #print(batch_features_ext)

        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
def compute_accuracy_per_class(test_features, test_label, test_id, test_attributes,cos_sim = False):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0
    #print(test_features.size())
    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]
    per_class_num = np.zeros(class_num)
    per_class_true= np.zeros(class_num)

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)

        if cos_sim:
            relations = F.cosine_similarity(batch_features_ext, sample_features_ext).view(-1, class_num)
        else:
            relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels_id, batch_per_num = np.unique(re_batch_labels , return_counts=True) 
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
        
        for each in range(re_batch_labels_id.size):
            #print(re_batch_labels_id[each])
            #print(batch_per_num[each])
            per_class_num[re_batch_labels_id[each]] = per_class_num[re_batch_labels_id[each]] + batch_per_num[each]
        #print(re_batch_labels_id)
        #print('-'*100)
        #print(batch_per_num)
        #print('-'*100)


        _, predict_labels = torch.min(relations.data, 1)
        for j in range(batch_size):
            if predict_labels[j] == re_batch_labels[j]:
                per_class_true[re_batch_labels[j]] = per_class_true[re_batch_labels[j]] + 1
            
        
        
    per_accuracy = per_class_true[np.nonzero(per_class_num)] / per_class_num[np.nonzero(per_class_num)]
    

    test_accuracy = np.sum(per_accuracy)/1.0/np.count_nonzero(per_class_num)
    
    #print(np.count_nonzero(per_class_num))
    return  test_accuracy

In [8]:
print("training...")
last_accuracy = 0.0
loss_zero_num = 0
writer = SummaryWriter()
for episode in range(EPISODE):
    #attribute_network.train()
    attribute_network_scheduler.step(episode)
    metric_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    batch_id = np.unique(batch_labels)

    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_id]).squeeze(1)
    batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(batch_id.size, 0))
    batch_attributes_ext = batch_attributes.repeat(BATCH_SIZE, 1)

    batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*2048
    batch_attributes_ext = Variable(batch_attributes_ext).cuda(GPU)
    

    re_batch_labels = []
    for label in batch_labels.numpy():
        index = np.argwhere(batch_id == label)
        re_batch_labels.append(index[0][0])
    re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
    re_batch_labels = Variable(re_batch_labels).cuda(GPU)
    

    criterion = HardTripletLoss_D(margin = Margin).cuda(GPU)
    triplet_loss= criterion(metric_network(attribute_network(batch_attributes_ext)), 
                            metric_network(batch_features_ext), re_batch_labels)
    if triplet_loss == 0:
        loss_zero_num = loss_zero_num + 1
    metric_network.zero_grad()
    attribute_network.zero_grad()
    
    triplet_loss.backward()
    
    attribute_network_optim.step()
    metric_network_optim.step()
    
    if (episode+1)%200 == 0 or episode==0:
        print("episode:", episode+1, "loss", triplet_loss)
        print('loss_zero_number= ',loss_zero_num)
        #writer.add_scalar('data/loss_zero_number', loss_zero_num, episode)
        loss_zero_num = 0
        writer.add_scalar('data/loss', triplet_loss, episode)
    if (episode+1)%200 == 0 or episode==0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy_per_class(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy_per_class(test_features, test_label, np.arange(200), attributes)
        gzsl_seen_accuracy  = compute_accuracy_per_class(test_seen_features, test_seen_label, np.arange(200), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        #print('zsl:', zsl_accuracy2)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        #print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy2 , gzsl_seen_accuracy2, H2))
        print('_'*100)
        writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        writer.add_scalar('data/H', H, episode)
#writer.export_scalars_to_json("./test.json")
writer.close()

training...
episode: 1 loss tensor(3.1329, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.02066666666666667
gzsl: unseen=0.0000 , seen=0.0067 , h=0.0000
____________________________________________________________________________________________________
episode: 200 loss tensor(3.0009, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.09961168907071888
gzsl: unseen=0.0886 , seen=0.0961 , h=0.0922
____________________________________________________________________________________________________
episode: 400 loss tensor(2.9871, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.14898157023183325
gzsl: unseen=0.1106 , seen=0.1961 , h=0.1414
____________________________________________________________________________________________________
episode: 600 loss tensor(2.9600, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
cla

episode: 5600 loss tensor(2.0598, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5224651593751757
gzsl: unseen=0.4242 , seen=0.5023 , h=0.4600
____________________________________________________________________________________________________
episode: 5800 loss tensor(2.0321, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5265635449601254
gzsl: unseen=0.4338 , seen=0.4996 , h=0.4644
____________________________________________________________________________________________________
episode: 6000 loss tensor(1.9230, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.51705295364333
gzsl: unseen=0.4167 , seen=0.5057 , h=0.4569
____________________________________________________________________________________________________
episode: 6200 loss tensor(1.9550, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200

episode: 11200 loss tensor(1.5333, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5427315123378509
gzsl: unseen=0.4431 , seen=0.5245 , h=0.4804
____________________________________________________________________________________________________
episode: 11400 loss tensor(2.2160, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5494967217565323
gzsl: unseen=0.4483 , seen=0.5263 , h=0.4842
____________________________________________________________________________________________________
episode: 11600 loss tensor(1.5949, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5555781582939627
gzsl: unseen=0.4476 , seen=0.5266 , h=0.4839
____________________________________________________________________________________________________
episode: 11800 loss tensor(1.8577, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class nu

episode: 16800 loss tensor(1.7257, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5430453140006817
gzsl: unseen=0.4496 , seen=0.5283 , h=0.4858
____________________________________________________________________________________________________
episode: 17000 loss tensor(2.1651, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5543294474787368
gzsl: unseen=0.4590 , seen=0.5239 , h=0.4893
____________________________________________________________________________________________________
episode: 17200 loss tensor(1.5144, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5429892157032635
gzsl: unseen=0.4502 , seen=0.5324 , h=0.4878
____________________________________________________________________________________________________
episode: 17400 loss tensor(1.8845, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class nu

episode: 22400 loss tensor(0.5548, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5365777123133806
gzsl: unseen=0.4485 , seen=0.5268 , h=0.4845
____________________________________________________________________________________________________
episode: 22600 loss tensor(1.3041, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5322501993600944
gzsl: unseen=0.4421 , seen=0.5351 , h=0.4842
____________________________________________________________________________________________________
episode: 22800 loss tensor(1.4253, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5418081446462113
gzsl: unseen=0.4470 , seen=0.5260 , h=0.4833
____________________________________________________________________________________________________
episode: 23000 loss tensor(1.5743, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class nu

...

episode: 77800 loss tensor(0., device='cuda:0')
loss_zero_number=  127
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.49916827388763463
gzsl: unseen=0.3823 , seen=0.5159 , h=0.4392
____________________________________________________________________________________________________
episode: 78000 loss tensor(0., device='cuda:0')
loss_zero_number=  110
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.502128672460791
gzsl: unseen=0.3908 , seen=0.5279 , h=0.4491
____________________________________________________________________________________________________
episode: 78200 loss tensor(1.00000e-02 *
       5.8328, device='cuda:0')
loss_zero_number=  122
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.48505393020059934
gzsl: unseen=0.3693 , seen=0.5274 , h=0.4344
____________________________________________________________________________________________________
episode: 78400 loss tensor(0.1421, device='cuda:0')
loss_zero_number=  139
Testing...


episode: 83200 loss tensor(0., device='cuda:0')
loss_zero_number=  140
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.49429277630132806
gzsl: unseen=0.3748 , seen=0.5200 , h=0.4356
____________________________________________________________________________________________________
episode: 83400 loss tensor(0.1289, device='cuda:0')
loss_zero_number=  152
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.47654370446920535
gzsl: unseen=0.3622 , seen=0.5285 , h=0.4298
____________________________________________________________________________________________________
episode: 83600 loss tensor(0., device='cuda:0')
loss_zero_number=  132
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.48358674729289836
gzsl: unseen=0.3659 , seen=0.5162 , h=0.4282
____________________________________________________________________________________________________
episode: 83800 loss tensor(0., device='cuda:0')
loss_zero_number=  150
Testing...
class num: 50
class num

class num: 200
class num: 200
zsl: 0.49011367090907465
gzsl: unseen=0.3661 , seen=0.5195 , h=0.4295
____________________________________________________________________________________________________
episode: 88800 loss tensor(0., device='cuda:0')
loss_zero_number=  154
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4888797299646126
gzsl: unseen=0.3664 , seen=0.5245 , h=0.4314
____________________________________________________________________________________________________
episode: 89000 loss tensor(1.00000e-02 *
       6.7990, device='cuda:0')
loss_zero_number=  166
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4896256828207944
gzsl: unseen=0.3693 , seen=0.5253 , h=0.4337
____________________________________________________________________________________________________
episode: 89200 loss tensor(0., device='cuda:0')
loss_zero_number=  147
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.48986613714021054
gzsl: unseen=0.3694 , seen=0.52

episode: 94200 loss tensor(0., device='cuda:0')
loss_zero_number=  155
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4998225686234045
gzsl: unseen=0.3731 , seen=0.5329 , h=0.4389
____________________________________________________________________________________________________
episode: 94400 loss tensor(0.1656, device='cuda:0')
loss_zero_number=  162
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4995973490594918
gzsl: unseen=0.3715 , seen=0.5257 , h=0.4353
____________________________________________________________________________________________________
episode: 94600 loss tensor(0., device='cuda:0')
loss_zero_number=  161
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4987936026755685
gzsl: unseen=0.3714 , seen=0.5327 , h=0.4377
____________________________________________________________________________________________________
episode: 94800 loss tensor(0., device='cuda:0')
loss_zero_number=  145
Testing...
class num: 50
class num: 2

episode: 99800 loss tensor(0., device='cuda:0')
loss_zero_number=  164
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.491933967252748
gzsl: unseen=0.3687 , seen=0.5279 , h=0.4342
____________________________________________________________________________________________________
episode: 100000 loss tensor(0., device='cuda:0')
loss_zero_number=  171
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4919995134279907
gzsl: unseen=0.3653 , seen=0.5237 , h=0.4304
____________________________________________________________________________________________________
